# Workshop 4 (Week beginning April 20)
# Quality control for sequencing data

Suppose you had a biological question of interest that could be solved by sequencing. You've received some funding to perform a sequencing run (lucky you!). The sample is run in a sequencing machine (you know a few kinds of sequencers by now) and you are delivered a file containing all of your reads. It is possible that there were problems in the sequencer or the starting material used for sequencing (DNA). Before you perform your analysis and report your findings, you want to be sure that the data you have is of good quality. This is where quality control (QC) comes in.

This week you will write your own code to perform QC on some real sequencing data.

## Task 1 - Reading in the read set

First, we read in some sample reads. These reads were selected from the ERR024571 read set.

In [ ]:
import skbio

fname = 'data/ERR024571_selected.fastq' 
registry = skbio.io.read(fname, format = 'fastq', phred_offset = 33)
readset = list(registry)
len(readset)

## Task 2 - Compute per sequence qualities

The per sequence quality is the average of the base qualities in a sequence.
Review the "DNA Sequencing and short reads" lecture to remind yourself of what these quality scores mean.

In [ ]:
# Display the quality scores for the first 10 bases of the first read.
r1_meta = readset[0].positional_metadata
r1_meta.head(10)

### Crash course on `pandas` and `numpy`

The output of the code above is a data frame object. Most data can be stored in data frames and many functions to process them are available in the `pandas` library. We will first try out a few indexing operations and write up a function to compute the per sequence quality.

If you are new to `pandas`, this tutorial below will help. It is also available under the additional resources module on the LMS.
> http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

In [ ]:
import pandas as pd
import numpy as np

# All of these indexes give the same results. Think of the data frame as a 2-D matrix.
# Here we index using positions.

print(r1_meta.iloc[:3,])
print(r1_meta.iloc[0:3, 0:1])
print(r1_meta.iloc[0:3, :])
print(r1_meta.iloc[0:3,])

Above we use information on the position along the data frame (i.e. rows 0:3 and column 0:1). Instead of this, we could use *row names* and *column names* to index. In the example below, 0:2 refers to the rows with names '0', '1', and '2' and the column with the name 'quality'. Since we slice the columns and select just one, what we get is a `numpy` array.

A simple way to reason about these `numpy` arrays is to think of them as vectors and matrices. A vector or matrix with named rows and columns is a `pandas` data frame.

In [ ]:
# This gives the same result but returns an array rather than a data frame.
# Here we index using names - 0, 1, and 2 are row names rather than positions.
print(r1_meta.loc[0:2, 'quality'])

In [ ]:
# Retrieve the base quality at position 6.
r1_qualities = r1_meta.loc[:, 'quality']
r1_qualities[2]

We can now write a function to compute the sequence quality of a given sequence with a for loop.

In [ ]:
def read_quality_with_loop(read):
    """
    Compute the quality of a given read (DNA sequence).
    This is the average of base qualities.
    Return a single floating point number.
    """



In [ ]:
print(read_quality_with_loop(readset[0])) # should give 27.36

The same can be done more simply using functions from the `numpy` library. This implementation is also considerably faster.

In [ ]:
def read_quality(read):
    """
    Compute the quality of a given read (DNA sequence).
    This is the average of base qualities.
    Return a single number representing the quality
    """
    
    return np.mean(read.positional_metadata.loc[:, 'quality'])


In [ ]:
print(read_quality_with_loop(readset[0]))

## Task 3 - Compute per sequence quality distribution

We will compute per sequence qualities for all reads and compute a sequence quality histogram. This is done by binning the data. We will create bins of integers i.e. given an integer $i$ the interval is $[i, i + 1)$, that is all numbers between $i$ and $i + 1$ including $i$ but excluding $i + 1$.

We will create a dictionary using this. Keys in the dictionary will be $i$ and values will be the number of quality scores falling in the interval $[i, i + 1)$.

The `np.floor` function can be used to round down quality scores to the nearest integer.

In [ ]:
print(np.floor(30.01))
print(np.floor(30.99))

In [ ]:
def read_histogram_dict(reads):
    """
    Compute a dictionary representing the histogram
    Keys are the starting point of the interval
    Values are the number of items in the interval
    """


In [ ]:
read_histogram_dict(readset[0:4]) # should give {27.0: 2, 28.0: 1, 26.0: 1}

Let's get to plotting! 
We use the `matplotlib` library to create simple plots. You can learn more about `matplotlib` using the link below. The link is also available on the additional resources module on the LMS.
>https://matplotlib.org/tutorials/introductory/usage.html#sphx-glr-tutorials-introductory-usage-py

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

hist_data = read_histogram_dict(readset)
plt.scatter(hist_data.keys(), hist_data.values())
plt.xlabel('Phred quality score')
plt.ylabel('Frequency')
plt.show()

## Task 4 - Use FastQC to perform the analysis

The above analysis and more can be performed using the FastQC program. We will use this to analyse all of the forward reads (rather than just the selection we have been using up to this point). 

You can invoke a terminal command from a Jupyter notebook by preceeding it with an exclamation mark as in the cell below (or you can just type the command in the terminal directly).

You can access the help menu with the command `fastqc -h`.

Note: fastqc is already installed on the COMP90016 server. To install it on a personal device, use the conda command `conda install -c bioconda fastqc`.

In [ ]:
!fastqc -o ./ data/ERR024571_1.fastq.gz

The report can be viewed by right clicking the file (.html) and selecting *Open in New Browser Tab*

## Task 5 - What does this all mean?

The report information about several QC metrics.

You should refer to this documentation to help you figure out what some of the metrics mean: 
>https://dnacore.missouri.edu/PDF/FastQC_Manual.pdf

Try answering some of these questions:

1. How many reads are in the read set?
2. How long are the reads?
3. Are all the reads the same length?
4. How are the quality scores distributed across the length of the reads?
5. Are there any overrepresented sequences?
6. What is the likely source of the overrepresented sequeces?
7. Overall, is this read set of acceptable quality?


FastQC outputs a report for an individual FASTQ file. When dealing with multiple read sets, MultiQC provides a summary of all FastQC reports. See https://multiqc.info/ and have a look at the example reports.

This is just one layer of quality control. More QC would need to be done, depending on type of analysis. For example, if RNA-seq was performed for the purpose of differential expression analysis, further checks and filters would be applied such as filtering out of genes with low read counts. QC should form an important component of all analyses. Ultimately, it can save a lot of time and effort.


Thank you to Dr. Dieter Bulach and Dharmesh Bhuva for developing the tutorial material. Updated by Steven Morgan.